Learning to use decorator, getters, setters, properties. 

This uses the performance timing methods in the functools wraps module to create a @timeit decorator you can stick in front of any function to also return the time it took to run.

In [ ]:
from functools import wraps
import time

def timeit(func):
    @wraps(func)
    def timeit_wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        total_time = end_time - start_time
        print(f"Function {func.__name__}{args} {kwargs} took {total_time:.4f} seconds")
        return result
    return timeit_wrapper

In [4]:
from dataclasses import dataclass
from datetime import datetime

@dataclass
class Transaction:
    _sender: str
    _recipient: str
    _amount: float
    _date: datetime = None

    # Sender getter and setter

    @property
    def sender(self):
        return self._sender
    
    @sender.setter
    def sender(self, sender):
        if not isinstance(sender, str):
            raise TypeError("Sender must be a string")
        self._sender = sender

    # Recipient getter and setter

    @property
    def recipient(self):
        return self._recipient

    @recipient.setter
    def recipient(self, recipient):
        if not isinstance(recipient, str):
            raise TypeError("Recipient must be a string")
        self._recipient = recipient

    # Amount getter and setter

    @property
    def amount(self):
        return self._amount

    @amount.setter
    def amount(self, amount):
        if not isinstance(amount, float):
            raise TypeError("Amount must be a float")
        self._amount = amount

    # Date getter and setter and deleter

    @property
    def date(self):
        return self._date

    @date.setter
    def date(self, date):
        if date is not None and not isinstance(date, str):
            raise TypeError("Date must be a string or None")
        self._date = date

    @date.deleter
    def date(self):
        self._date = None




Now with pydantic: 

In [ ]:
# pip install pydantic

In pydantic, you don't usually use getters and setters, the validation happens when the object is instantiated. So here we weill build validators instead of getters and setters

In [3]:
from pydantic import BaseModel, Field, validator, ValidationError

class TransactionPydantic(BaseModel):
    sender: str
    recipient: str
    _amount: float
    _date: datetime | None # this is a type hint for python 3.10  if using 3.9 you have to Optional from typing

@validator("sender", "recipient")
def check_string(cls, v):
    if not isinstance(v, str):
        raise TypeError("Sender and recipient must be strings")
    return v

@validator("amount")
def check_amount(cls, v):
    if not isinstance(v, float):
        raise TypeError("Amount must be a float")
    return v

@validator("date")
def check_date(cls, v):
    if v is not None and not isinstance(v, datetime):
        raise TypeError("Date must be datetime or None")
    return v


For any object, you can do help(Counter) to see how it works

You can do a retry decorator that is useful:

In [3]:
from functools import wraps
def retry(func):
    @wraps(func)
    def retry_wrapper(*args, **kwargs):
        for i in range(3):
            try:
                result = func(*args, **kwargs)
                return result
            except Exception as e:
                print(f"Exception {e} occurred. Retrying...")
    return retry_wrapper

In [8]:
@retry
def bad_func():
    raise Exception("Bad func failed")

@retry
def good_func():
    return "Good func succeeded"


In [9]:
bad_func()
good_func()

Exception Bad func failed occurred. Retrying...
Exception Bad func failed occurred. Retrying...
Exception Bad func failed occurred. Retrying...


'Good func succeeded'

# Testing is another use case for decorators to add functionality  to tests

In PyTest, there are several built-in decorators you can use to modify the behavior of your test functions. Here are some common examples:

1. **`@pytest.mark.parametrize`:** This decorator allows you to run a test function multiple times with different arguments.

In this example below, `test_square` will be run 5 times, each time with a different pair of arguments.

In [ ]:
@pytest.mark.parametrize("input,expected_output", [(1, 1), (2, 4), (3, 9), (4, 16), (5, 25)])
def test_square(input, expected_output):
    assert input ** 2 == expected_output




2. **`@pytest.mark.skip`:** This decorator lets you skip a test. You can also provide a reason which will be shown when the tests are run.

    ```python
    @pytest.mark.skip(reason="No need to test this function right now.")
    def test_my_function():
        pass
    ```

3. **`@pytest.mark.xfail`:** Marks a test function as expected to fail. If the test passes, it will be reported as an "unexpected success".

    ```python
    @pytest.mark.xfail
    def test_my_function():
        assert 0
    ```

4. **`@pytest.fixture`:** This decorator lets you define setup code that should run before your tests. You can then include the fixture as an argument in your test functions, and PyTest will automatically call it for you.

    ```python
    @pytest.fixture
    def setup_data():
        return {"key": "value"}

    def test_my_function(setup_data):
        assert setup_data["key"] == "value"
    ```

    In this example, `setup_data` will be called before `test_my_function` and its return value will be passed as an argument.

These decorators are some of the ways you can modify the behavior of your tests in PyTest, but there are many other features and options available as well. You can learn more in the [PyTest documentation](https://docs.pytest.org/en/latest/).